In [1]:
import argparse

# Librerías principales
import pandas as pd
import numpy as np
from datetime import date
import random
from dateutil.relativedelta import relativedelta

# Librerías Proyecto
from lib.RecSysClusters import ClusterRFM
from lib.RecSys import AprioriSys

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm

In [3]:
df_recsys = pd.read_csv('../data/data_final/full_trx.csv')

In [4]:
subcats = df_recsys['subcat_comercial'].unique()

In [5]:
np.save('subcats.npy', subcats)

In [6]:
min_sup = 0.0005

# Ajustar algoritmo apriori
apriori_sys = AprioriSys(df=df_recsys[['venta_id_crp','subcat_comercial']], porduct_column='subcat_comercial', min_support=min_sup)
rules = apriori_sys.fit_rules()

In [10]:
rules.to_json('rules.json')

In [15]:
rules.iloc[0,1]

frozenset({'barquillo'})

In [22]:
rules[rules['consequents'] == frozenset({'yoghurt'})].sort_values('confidence', ascending=False).head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
211,(frutos_secos),(yoghurt),0.015921,0.007509,0.000836,0.052535,6.995841,0.000717,1.047522
894,"(bolleria, jugo)",(yoghurt),0.020578,0.007509,0.000614,0.029819,3.970852,0.000459,1.022995
221,(galleta_dulce),(yoghurt),0.031002,0.007509,0.000555,0.017908,2.384677,0.000322,1.010588
1989,"(jugo, panaderia)",(yoghurt),0.029614,0.007509,0.000522,0.017637,2.348625,0.000300,1.010309
271,(jugo),(yoghurt),0.094175,0.007509,0.001600,0.016987,2.262117,0.000893,1.009642
95,(bolleria),(yoghurt),0.210469,0.007509,0.002250,0.010690,1.423537,0.000669,1.003215
33,(alfajor),(yoghurt),0.060101,0.007509,0.000544,0.009055,1.205812,0.000093,1.001560
964,"(bolleria, panaderia)",(yoghurt),0.118986,0.007509,0.000972,0.008165,1.087331,0.000078,1.000661
321,(panaderia),(yoghurt),0.368446,0.007509,0.002860,0.007762,1.033630,0.000093,1.000255


## GRID SHEARCH: ARIORI CONTRA BENCHMARKS

In [14]:
rules.shape

(7324, 9)

In [22]:
# Predecir
resultados={
    'venta_id_crp':[],
    'apriori_metric':[],
    'apriori_min_support':[],
    #'n_recommendations':[],
    'rec_apriori_1':[],
    'rec_apriori_2':[],
    'rec_apriori_3':[],
    'rec_random_1':[],
    'rec_random_2':[],
    'rec_random_3':[],
    'rec_top_subcat_1':[],
    'rec_top_subcat_2':[],
    'rec_top_subcat_3':[]
}


for min_sup in [0.0005, 0.005, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045]:


    #Definir Subcategorias
    subcategorias = list(train['subcat_comercial'].unique())

    # Obtener lista de productos populares (top)
    df_top_n = train.groupby('subcat_comercial').count().sort_values('venta_id_crp', ascending=False)
    top = list(df_top_n.index)

    # Ajustar algoritmo apriori
    apriori_sys = AprioriSys(df=train[['venta_id_crp','subcat_comercial']], porduct_column='subcat_comercial', min_support=min_sup)
    rules = apriori_sys.fit_rules()

    #rules = pd.read_csv('rules.csv')

    # Iterar por el dataframe del cluster
    for index,row in tqdm(df_test.iterrows()):

        # Iterar por función a maximizar
        for metric in ['confidence', 'lift']:

            n_rec = 3

            # Obtener Recomendaciones de los modelos benchmark
            rec_random = random.sample(subcategorias,n_rec)
            top_n = top[:n_rec]

            # Obtener Recomendaciones del modelo apriori
            recomendacion_apriori = apriori_sys.predict(basket=pd.DataFrame(row).T, metric=metric, n_recommendations=n_rec)

            # Almacenar Resultados Especificos
            resultados['venta_id_crp'].append(index)
            resultados['apriori_metric'].append(metric)
            resultados['apriori_min_support'].append(min_sup)
            #resultados['n_recommendations'].append(n_rec)

            for n in range(n_rec):
                try:
                    resultados[f'rec_apriori_{n+1}'].append(recomendacion_apriori[f'rec_{n}'])
                except:
                    resultados[f'rec_apriori_{n+1}'].append(None)

                resultados[f'rec_random_{n+1}'].append(rec_random[n])
                resultados[f'rec_top_subcat_{n+1}'].append(top_n[n])

273788it [31:26, 145.16it/s]
273788it [28:42, 158.98it/s]
273788it [28:32, 159.89it/s]
273788it [28:18, 161.23it/s]
273788it [28:11, 161.86it/s]
273788it [28:14, 161.58it/s]
273788it [28:06, 162.35it/s]
273788it [28:29, 160.20it/s]
273788it [28:10, 161.99it/s]


In [23]:
#Generar el dataframe de los ejecrcicios
df = pd.DataFrame(resultados)
#for i in list_results:
#    df = df.append(pd.DataFrame(i), ignore_index=True)

#Exportar resultados en parquet


In [25]:
df.shape

(4928184, 12)

In [21]:
df.apriori_min_support.unique()

array([0.0002, 0.001 , 0.05  , 0.1   ])

In [26]:
df.to_parquet('resultados/resultados_finales_nov2022V2.parquet')

In [ ]:
df

In [24]:
leave_one = pd.read_csv('../data/leave_one.csv')
df = df.merge(leave_one, on='venta_id_crp')

In [25]:
df['hit_apriori'] = df.apply(lambda x: 1 if x['leave_one'] in x['rec_apriori'] else 0, axis=1)
df['hit_random'] = df.apply(lambda x: 1 if x['leave_one'] in x['rec_random'] else 0, axis=1)
df['hit_top'] = df.apply(lambda x: 1 if x['leave_one'] in x['rec_top_subcat'] else 0, axis=1)

In [27]:
df.query('hit_apriori == 1')

,venta_id_crp,apriori_metric,n_recommendations,rec_apriori,rec_random,rec_top_subcat,leave_one,hit_apriori,hit_random,hit_top
12,726911,confidence,1,[COCKTAIL CONGELADO],[QUICHE],[MAP],COCKTAIL CONGELADO,1,0,0
13,726911,confidence,2,"[COCKTAIL CONGELADO, MAP FAMILIAR]","[MAP FAMILIAR, ENTRADA FAMILIAR]","[MAP, PANADERIA]",COCKTAIL CONGELADO,1,0,0
14,726911,confidence,3,"[COCKTAIL CONGELADO, HELADO, MAP FAMILIAR]","[GASEOSA, POSTRE INDIVIDUAL, FRUTOS SECOS]","[MAP, PANADERIA, BOLLERIA]",COCKTAIL CONGELADO,1,0,0
15,726911,lift,1,[COCKTAIL CONGELADO],[MAR],[MAP],COCKTAIL CONGELADO,1,0,0
16,726911,lift,2,"[COCKTAIL CONGELADO, MAP FAMILIAR]","[ICE TEA, JUGO]","[MAP, PANADERIA]",COCKTAIL CONGELADO,1,0,0
...,...,...,...,...,...,...,...,...,...,...
5565,730931,lift,1,[GASEOSA],[MAP FAMILIAR],[MAP],GASEOSA,1,0,0
5566,730931,lift,2,"[GASEOSA, HELADO]","[KOMBUCHA, IMPULSO SALADO]","[MAP, PANADERIA]",GASEOSA,1,0,0
5567,730931,lift,3,"[AGUA, GASEOSA, HELADO]","[QUESO, CARNE ROJA, GNOCCHI]","[MAP, PANADERIA, BOLLERIA]",GASEOSA,1,0,0
5569,730936,confidence,2,"[MAP, POSTRE INDIVIDUAL]","[PLATO LIVIANO, KOMBUCHA]","[MAP, PANADERIA]",POSTRE INDIVIDUAL,1,0,0


In [19]:
df.to_csv('./resultados/apriori_sin_cluster_corregido.csv', index=False)

In [20]:
df['len_rec_apriori'] = df['rec_apriori'].apply(lambda x: len(x))

In [21]:
#df.sort_values('len_rec_apriori', ascending=False)

In [22]:
df.groupby(['apriori_metric', 'n_recommendations'])[['hit_apriori', 'hit_random', 'hit_top']].mean()

hit_apriori  hit_random   hit_top
apriori_metric n_recommendations                                   
confidence     1                     0.267783    0.014508  0.155877
               2                     0.386704    0.028428  0.254511
               3                     0.452305    0.044280  0.309313
lift           1                     0.117746    0.014261  0.155877
               2                     0.216427    0.029216  0.254511
               3                     0.295686    0.044713  0.309313

In [24]:
df.query('len_rec_apriori == 0')

,venta_id_crp,apriori_metric,n_recommendations,rec_apriori,rec_random,rec_top_subcat,leave_one,hit_apriori,hit_random,hit_top,len_rec_apriori
6744,731786,confidence,1,[],[QUICHE],[MAP],GALLETA DULCE,0,0,0,0
6745,731786,confidence,2,[],"[MAP, JAMON]","[MAP, PANADERIA]",GALLETA DULCE,0,0,0,0
6746,731786,confidence,3,[],"[COCKTAIL CONGELADO, BARRA DE CEREAL, JUGO]","[MAP, PANADERIA, BOLLERIA]",GALLETA DULCE,0,0,0,0
6747,731786,lift,1,[],[DIP],[MAP],GALLETA DULCE,0,0,0,0
6748,731786,lift,2,[],"[IMPULSO SALADO, MAR]","[MAP, PANADERIA]",GALLETA DULCE,0,0,0,0
6749,731786,lift,3,[],"[MAR, EMPANADITA, PISCO SOUR]","[MAP, PANADERIA, BOLLERIA]",GALLETA DULCE,0,0,0,0


In [4]:
random.seed(20220720)

list_results = []
top_n_clusters = []

for rolling in [False]:
    
    start_date = date(2020,3,1)
    max_date =  date(2022,5,1)
    minimum_windows_size = 12
    
    print(f'rolling {rolling}')
    
    start_train = start_date
    end_train = start_train + relativedelta(months=minimum_windows_size)
    start_test = start_train + relativedelta(months=minimum_windows_size+1)
    end_test = start_test + relativedelta(months=1)

    while start_test <= max_date:

        print(start_train, end_train, start_test, end_test)

        # Definir DataFrames
        df_cluster_train = df_clusters[(df_clusters['year_month'] >= start_train.strftime('%Y-%m-%d')) & (df_clusters['year_month'] <= end_train.strftime('%Y-%m-%d'))].reset_index(drop=True)
        df_cluster_test = df_clusters[(df_clusters['year_month'] >= (end_train + relativedelta(months=1)).strftime('%Y-%m-%d')) & (df_clusters['year_month'] < (end_train + relativedelta(months=2)).strftime('%Y-%m-%d'))].reset_index(drop=True)
        df_recsys_train = df_recsys[(df_recsys['fecha'] >= start_train.strftime('%Y-%m-%d')) & (df_recsys['fecha'] <= end_train.strftime('%Y-%m-%d'))].reset_index(drop=True)
        df_recsys_test = df_test[(df_test['fecha'] >= start_test.strftime('%Y-%m-%d')) & (df_test['fecha'] < end_test.strftime('%Y-%m-%d'))].reset_index(drop=True)
        
        #Definir Subcategorias
        subcategorias = list(df_recsys_train['subcat_comercial'].unique())
                
        # Definir Clusters de Entrenamiento
        cluster = ClusterRFM(df=df_cluster_train, user_id='usuario_id_crp', frequency_column='freq_all_1', monetary_column='mon_all_1', n_clusters=3)
        cluster_cliente_entrenamiento = cluster.cluster_customer()

        # Predecir Clusters de Test
        cluster_test = ClusterRFM(df=df_cluster_test, user_id='usuario_id_crp', frequency_column='freq_all_1', monetary_column='mon_all_1', n_clusters=3, trained_model=cluster.trained_model, trained_scaled_model=cluster.scaler)
        cluster_cliente_test = cluster_test.cluster_customer()

        # Asoignar cluster a cada cliente
        df_recsys_train['cluster'] = df_recsys_train['usuario_id_crp'].map(cluster_cliente_entrenamiento)
        df_recsys_test['cluster'] = df_recsys_test['usuarioidcrp'].map(cluster_cliente_test)
        
        # Definir dataframes por cluster
        set_df_recsys_train = {}
        set_df_recsys_test = {}
        set_top_n_train = {}
        listas_top_n_train = {}
        for n_cluster in range(cluster.n_clusters):
            set_df_recsys_train[f'cluster_{n_cluster}'] = df_recsys_train[df_recsys_train['cluster'] == n_cluster].drop(columns='cluster',axis=1).reset_index(drop=True)
            set_df_recsys_test[f'cluster_{n_cluster}'] = df_recsys_test[df_recsys_test['cluster'] == n_cluster].drop(columns='cluster',axis=1).set_index('ventaidcrp')
            
            # Obtener lista de productos populares (top) del cluster y almacenar
            df_top_n = set_df_recsys_train[f'cluster_{n_cluster}'].reset_index(drop=True)
            df_top_n = df_top_n.groupby('subcat_comercial').count().sort_values('venta_id_crp', ascending=False)
            top = list(df_top_n.index)
            listas_top_n_train[n_cluster] = top
            
            
        # Ejecutar Ejercicio por Cluster
        cluster_rules = {}
        for n_cluster in range(cluster.n_clusters):
            
            # Ajustar algoritmo apriori
            apriori_sys = AprioriSys(df=set_df_recsys_train[f'cluster_{n_cluster}'], porduct_column='subcat_comercial', min_support=0.001)
            cluster_rules[f'reglas_cluster_{n_cluster}'] = apriori_sys.fit_rules()
            
            # Predecir
            resultados={
                'rolling':[],
                'start_train':[],
                'end_train':[],
                'start_test':[],
                'end_test':[],
                'venta_id_crp':[],
                'cluster':[],
                'apriori_metric':[],
                'n_recommendations':[],
                'rec_apriori':[],
                'rec_random':[],
                'rec_top_subcat':[]
            }
            
            # Iterar por el dataframe del cluster
            for index,row in tqdm(set_df_recsys_test[f'cluster_{n_cluster}'].iterrows()):
                
                # Iterar por función a maximizar
                for metric in ['consequent support', 'confidence', 'lift', 'leverage', 'conviction']:
                    
                    # Iterar por cantidad de productos recomendados
                    for n_rec in [3]:
                        
                        # Obtener Recomendaciones de los modelos benchmark
                        rec_random = random.sample(subcategorias,n_rec)
                        top_n = listas_top_n_train[n_cluster][:n_rec]
                        
                        # Obtener Recomendaciones del modelo apriori
                        recomendacion_apriori = apriori_sys.predict(basket=pd.DataFrame(row[2:]).T, metric=metric, n_recommendations=n_rec)
                        
                        # Almacenar Resultados Especificos
                        resultados['rolling'].append(rolling)
                        resultados['start_train'].append(start_train)
                        resultados['end_train'].append(end_train)
                        resultados['start_test'].append(start_test)
                        resultados['end_test'].append(end_test)
                        resultados['venta_id_crp'].append(index)
                        resultados['cluster'].append(n_cluster)
                        resultados['apriori_metric'].append(metric)
                        resultados['n_recommendations'].append(n_rec)
                        resultados['rec_apriori'].append(recomendacion_apriori)
                        resultados['rec_random'].append(rec_random)
                        resultados['rec_top_subcat'].append(top_n)
            
            # Almacenar Resultados Generales
            list_results.append(resultados)

        # Actualizar Fechas
        if rolling == True:
            start_train += relativedelta(months=1)

        end_train += relativedelta(months=1)
        start_test += relativedelta(months=1)
        end_test += relativedelta(months=1)
        

#Generar el dataframe de los ejecrcicios
df = pd.DataFrame()
for i in list_results:
    df = df.append(pd.DataFrame(i), ignore_index=True)

#Exportar resultados en parquet
df.to_csv('./resultados/recursive_v2.csv')

rolling False
2020-03-01 2021-03-01 2021-04-01 2021-05-01


7016it [10:19, 11.32it/s]
3095it [04:53, 10.53it/s]
1589it [01:50, 14.34it/s]


2020-03-01 2021-04-01 2021-05-01 2021-06-01


2843it [04:35, 10.32it/s]
1485it [01:43, 14.28it/s]
6277it [09:11, 11.38it/s]


2020-03-01 2021-05-01 2021-06-01 2021-07-01


...

2538it [03:49, 11.04it/s]
1959it [02:13, 14.62it/s]
4531it [06:24, 11.77it/s]


2020-03-01 2022-03-01 2022-04-01 2022-05-01


3308it [04:44, 11.63it/s]
1906it [02:10, 14.65it/s]
6023it [08:31, 11.77it/s]


2020-03-01 2022-04-01 2022-05-01 2022-06-01


7005it [09:54, 11.79it/s]
2018it [02:17, 14.63it/s]
2983it [04:20, 11.47it/s]


In [21]:
pd.read_json('rules.json')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,[AGUA],[BARQUILLO],0.014629,0.096918,0.002019,0.138031,1.424216,0.000601,1.047698
1,[BARQUILLO],[AGUA],0.096918,0.014629,0.002019,0.020834,1.424216,0.000601,1.006338
2,[AGUA],[ENSALADA],0.014629,0.058156,0.001960,0.133955,2.303347,0.001109,1.087522
3,[ENSALADA],[AGUA],0.058156,0.014629,0.001960,0.033695,2.303347,0.001109,1.019731
4,[AGUA],[GASEOSA],0.014629,0.060747,0.002394,0.163658,2.694106,0.001505,1.123049
...,...,...,...,...,...,...,...,...,...
16943,[COCKTAIL CONGELADO],"[QUESO, PANADERIA, GALLETA SALADA, DIP, CHARCU...",0.101646,0.001789,0.000814,0.008005,4.473976,0.000632,1.006266
16944,[QUESO],"[COCKTAIL CONGELADO, PANADERIA, GALLETA SALADA...",0.106170,0.000954,0.000814,0.007664,8.031252,0.000712,1.006761
16945,[GALLETA SALADA],"[COCKTAIL CONGELADO, QUESO, PANADERIA, DIP, CH...",0.070417,0.001286,0.000814,0.011555,8.981560,0.000723,1.010388
16946,[DIP],"[COCKTAIL CONGELADO, QUESO, PANADERIA, GALLETA...",0.048026,0.001815,0.000814,0.016942,9.335637,0.000726,1.015388


## Pruebas Random

In [35]:
comparacion = pd.read_json('../data/dataset_test.json')

In [36]:
comparacion_2 = comparacion.merge(resultados_2, how='inner', on='venta_id_crp')
comparacion_2

,fecha,venta_id_crp,usuario_id_crp,canasta_original,canasta_test,leave_one,recomendacion
0,2021-04-01,428056,1925806,"[POSTRE INDIVIDUAL, SANDWICH, JUGO]","[POSTRE INDIVIDUAL, SANDWICH]",JUGO,"[BARQUILLO, EMPANADA INDIVIDUAL, EMPANADITA, G..."
1,2021-04-01,428057,771075,"[EMPANADITA, BARQUILLO, PANADERIA, MAP, ALFAJOR]","[EMPANADITA, BARQUILLO, ALFAJOR, PANADERIA]",MAP,"[BOLLERIA, CALUGA, FRUTOS SECOS, MAP, POSTRE I..."
2,2021-04-01,428058,1853170,"[EMPANADITA, POSTRE FAMILIAR]",[POSTRE FAMILIAR],EMPANADITA,"[COCKTAIL CONGELADO, LASAÑA, MAP FAMILIAR, PAN..."
3,2021-04-01,428060,1916418,"[POSTRE INDIVIDUAL, PLATO LIVIANO, PANADERIA]","[POSTRE INDIVIDUAL, PLATO LIVIANO]",PANADERIA,"[EMPANADA INDIVIDUAL, EMPANADITA, ENSALADA, GA..."
4,2021-04-01,428068,526798,"[EMPANADITA, PIZZA, PANADERIA]","[EMPANADITA, PIZZA]",PANADERIA,"[BOLLERIA, EMPANADA INDIVIDUAL, PANADERIA, POS..."
...,...,...,...,...,...,...,...
7011,2021-04-30,457941,1396669,"[JUGO, QUESO LAMINADO, PANADERIA]","[QUESO LAMINADO, PANADERIA]",JUGO,"[ALFAJOR, BOLLERIA, DIP, FRUTOS SECOS, JAMON, ..."
7012,2021-04-30,457948,677920,"[GALLETA DULCE, ALFAJOR, EMPANADA INDIVIDUAL, ...","[ALFAJOR, EMPANADA INDIVIDUAL, FRUTOS SECOS]",GALLETA DULCE,"[BARQUILLO, BOLLERIA, CALUGA, GALLETA SALADA, ..."
7013,2021-04-30,457953,1528614,"[PANADERIA, SALAME, QUESO LAMINADO]","[SALAME, PANADERIA]",QUESO LAMINADO,"[BOLLERIA, FRUTOS SECOS, JAMON, QUESO LAMINADO]"
7014,2021-04-30,457961,1477438,"[PANADERIA, GALLETA SALADA, MANTEQUILLA]","[MANTEQUILLA, PANADERIA]",GALLETA SALADA,"[BOLLERIA, JAMON, POSTRE INDIVIDUAL, QUESO LAM..."


In [37]:
res = []
for index,row in tqdm(comparacion_2.iterrows()):
   
    res.append(row['leave_one'] in row['recomendacion'])

comparacion_2['hit'] = res

7016it [00:00, 23149.25it/s]


In [38]:
sum(res) / comparacion_2.shape[0]

0.4605188141391106

In [32]:
sum(res) / comparacion_2.shape[0]

0.4484036488027366

In [26]:
sum(res) / comparacion_2.shape[0]

0.3983751425313569

In [39]:
comparacion_2

,fecha,venta_id_crp,usuario_id_crp,canasta_original,canasta_test,leave_one,recomendacion,hit
0,2021-04-01,428056,1925806,"[POSTRE INDIVIDUAL, SANDWICH, JUGO]","[POSTRE INDIVIDUAL, SANDWICH]",JUGO,"[BARQUILLO, EMPANADA INDIVIDUAL, EMPANADITA, G...",True
1,2021-04-01,428057,771075,"[EMPANADITA, BARQUILLO, PANADERIA, MAP, ALFAJOR]","[EMPANADITA, BARQUILLO, ALFAJOR, PANADERIA]",MAP,"[BOLLERIA, CALUGA, FRUTOS SECOS, MAP, POSTRE I...",True
2,2021-04-01,428058,1853170,"[EMPANADITA, POSTRE FAMILIAR]",[POSTRE FAMILIAR],EMPANADITA,"[COCKTAIL CONGELADO, LASAÑA, MAP FAMILIAR, PAN...",False
3,2021-04-01,428060,1916418,"[POSTRE INDIVIDUAL, PLATO LIVIANO, PANADERIA]","[POSTRE INDIVIDUAL, PLATO LIVIANO]",PANADERIA,"[EMPANADA INDIVIDUAL, EMPANADITA, ENSALADA, GA...",True
4,2021-04-01,428068,526798,"[EMPANADITA, PIZZA, PANADERIA]","[EMPANADITA, PIZZA]",PANADERIA,"[BOLLERIA, EMPANADA INDIVIDUAL, PANADERIA, POS...",True
...,...,...,...,...,...,...,...,...
7011,2021-04-30,457941,1396669,"[JUGO, QUESO LAMINADO, PANADERIA]","[QUESO LAMINADO, PANADERIA]",JUGO,"[ALFAJOR, BOLLERIA, DIP, FRUTOS SECOS, JAMON, ...",True
7012,2021-04-30,457948,677920,"[GALLETA DULCE, ALFAJOR, EMPANADA INDIVIDUAL, ...","[ALFAJOR, EMPANADA INDIVIDUAL, FRUTOS SECOS]",GALLETA DULCE,"[BARQUILLO, BOLLERIA, CALUGA, GALLETA SALADA, ...",False
7013,2021-04-30,457953,1528614,"[PANADERIA, SALAME, QUESO LAMINADO]","[SALAME, PANADERIA]",QUESO LAMINADO,"[BOLLERIA, FRUTOS SECOS, JAMON, QUESO LAMINADO]",True
7014,2021-04-30,457961,1477438,"[PANADERIA, GALLETA SALADA, MANTEQUILLA]","[MANTEQUILLA, PANADERIA]",GALLETA SALADA,"[BOLLERIA, JAMON, POSTRE INDIVIDUAL, QUESO LAM...",False


In [54]:
np.unique(lista_b)

array(['BARQUILLO', 'EMPANADITA', 'HELADO', 'PANADERIA',
       'POSTRE INDIVIDUAL', 'SOPAS'], dtype='<U17')